<a href="https://colab.research.google.com/github/surensnyper/ys_project_001/blob/main/3_Evaluated_Copy_of_Finetuned_Keras_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-trained Model

1.   video : https://www.youtube.com/watch?v=LuApA264Wbs&t=947s
2.   code : https://github.com/kimwoonggon/publicservant_AI/blob/master/(Uncased_Squad_V1_1)_%EC%BC%80%EB%9D%BC%EC%8A%A4%EB%A1%9C_Q%26A_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb

In [13]:
import os
import zipfile
import shutil

# BERT Model Download
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

# bert folder construction in the Colab
if "bert" not in os.listdir():
  os.makedirs("bert")
else:
  pass

--2022-01-30 21:11:01--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 172.253.123.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   261MB/s    in 1.5s    

2022-01-30 21:11:02 (261 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [17]:
# Unzip the Model zip file in the bert folder
bert_zip  = zipfile.ZipFile('uncased_L-12_H-768_A-12.zip')
bert_zip.extractall('bert')

bert_zip.close()

In [20]:
# Functoin Construction for copying the folder
def copytree(src, dst, symlinks=False, ignore=None):
  for item in os.listdir(src):
    s = os.path.join(src, item)
    d = os.path.join(dst, item)
    if os.path.isdir(s):
      shutil.copytrees(s, d, symlinks, ignore)
    else:
      shutil.copy2(s, d)

In [21]:
# The unzipped folder is "bert/uncased_L-12_H-768_A-12", but copying & moving the model to the bert folder for easier analysis
copytree("bert/uncased_L-12_H-768_A-12", "bert")

In [14]:
# Google drive and Colab Connecting
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
# Required modules are imported, e.g. Tensorflow, Pandas, Numpy, Keras etc
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers
from keras.layers import Layer

import codecs
from tqdm import tqdm
import shutil
import json

In [30]:
# To make various logs nor popped up
import warnings
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_WIN_LOG_LEVEL']='3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) # Version updated (tensorflow logging tf.logging -> tf.compat.v1.logging)

In [9]:
# A module 'keras-bert' install, for using bert easily in keras
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=e7de3771803f88cd08d1f2d4952f249363653396c8bcd6f9c681621b35883876
  Stored in directory: /root/.cache/pip/wheels/a4/e8/45/842b3a39831261aef9154b907eacbc4ac99499a99ae829b06f
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=20386c41715347432a05e698faa49454840553e48b4a8b658745f72d835e6ee8
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=d7115ab1bc5c2ec5c986acd4eaeb5badd1f77686ecdda9d0e943ef423f7d1c3d
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=6ea053ff25328a1960a2989c9ae2ddca2c415f2db8cb3bf118513d4d

In [6]:
# An optimizer 'keras-radam' import, which is an updated optimizer of 'Adam'
# 'radam' is to update learning rate from 0 to target value 'gradually', so training session is more stable
!pip install keras-radam

  Created wheel for keras-radam: filename=keras_radam-0.15.0-py3-none-any.whl size=14686 sha256=f77b91e909badfd21f95e2e4d1183e33b7d8e77d1d90be544da7fac25dfc58aa
  Stored in directory: /root/.cache/pip/wheels/23/6a/5f/d674f3b7b4d504b03148abd675e3703ba00c31763c04a2fc20
Successfully built keras-radam


In [10]:
# Other libraries for using bert model in the keras-bert libraries
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam.training import RAdamOptimizer # Original RAdam changed to RAdamOptimizer
# from keras_radam import RAdam
# https://github.com/CyberZHG/keras-radam (KERAS-RADAM)

In [15]:
# Hyperparameters set up

# Input sequence maximum length setup (Question + Context)
SEQ_LEN = 384

# Batch size (training size at one time)
BATCH_SIZE = 12

EPOCH = 2

# Learning rate is set in very small number like 1.5e-5.
# In fine tuning session, Learning rate must be very small 
# (Optimizer RAdam starts from LR 0 to 1.5e-5)
LR = 1.5e-5

# Model location designation
pretrained_path = "bert"
# Pretrained hyperparameters like weights
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
# Word corpus file location designation
vocab_path =  os.path.join(pretrained_path, 'vocab.txt')

# Configuration file designation
config_path = os.path.join(pretrained_path, 'bert_config.json')

# Column name of context
DATA_COLUMN = "context"
# Column name of question
QUESTION_COLUMN = "question"
# Column name of answer ('text')
TEXT = "text"

In [66]:
# Dictionary Construction for indexing each word
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
  for line in reader:
    token = line.strip()
    token_dict[token] = len(token_dict)

In [67]:
# Tokenizer Construction
tokenizer = Tokenizer(token_dict)

In [22]:
# Load the Pretrained Model
model = load_trained_model_from_checkpoint (
    config_path,
    checkpoint_path,
    training = False,
    trainable = True,
    seq_len = SEQ_LEN
)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 384)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 384)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 384, 768),  23440896    ['Input-Token[0][0]']            
 g)                              (30522, 768)]                                                    
                                                                                                  
 Embedding-Segment (Embedding)  (None, 384, 768)     1536        ['Input-Segment[0][0]']    

In [24]:
# For Transfer Learning, Customized Layer needs to be added after 12 Encoder
# by defining 'Non-masking' function, BERT Model's masked tensors disclosed

class NonMasking(Layer):
  def __init__(self, **kwargs):
    self.supports_masking = True
    super(NonMasking, self).__init__(**kwargs)

  def build(self, input_shape):
    input_shape = input_shape
  
  def compute_mask(self, input, input_mask = None):
    return None

  def call(self, x, mask = None):
    return x

  def get_output_shape_for(self, input_shape):
    return input_shape

In [28]:
# Activation function definition
def bert_gelu(x):
  """Gaussian Error Linear Unit.
  This is a smoother version of the RELU.
  Original paper: https://arxiv.org/abs/1606.08415
  Args:
    x : float Tensor to perform activation
  Returns:
    'x' with the GELU activation applied.
  """
  cdf = 0.5*(1.0+ K.tanh(
      (np.sqrt(2/np.pi)*(x+0.044715 * K.pow(x,3)))))
  return x*cdf

In [26]:
# Activation Function Layer attached to Transformer
class Start_End_Prediction(Layer):
  def __init__(self, seq_len, **kwargs):
    self.seq_len = SEQ_LEN
    self.supports_masking = True
    super(Start_End_Prediction, self).__init__(**kwargs)

  def build(self, input_shape):
    # A tensor ('self.W') multiplied with the final layer (12 encoder, batch_size, 384, 768)
    # Making Output tensor as (384, 2 dimension (768->2))
    self.W = self.add_weight(name='kernel',
                             shape = (input_shape[2],2),
                             initializer = 'uniform',
                             trainable = True)
    super(Start_End_Prediction, self).build(input_shape)

  def call(self, x):
    # Redifine Output dimension as (384 * 768)
    x = K.reshape(x, shape=(-1, self.seq_len, K.shape(x)[2]))
    # Dot production between self.W and x
    # (batch_size, 384, 768) * (384, 2) = (batch_size, 384, 2)
    x = K.dot(x, self.W)
    # (batch_size, 384, 2) -> (2, batch_size, 384)
    x = K.permute_dimensions(x, (2, 0, 1))

    # Split the (2, batch_size, 384) into 2 (batch_size, 384)  --> start_logits & end_logits
    # start_logits = (batch_size, 384)
    # end_logits = (batch_size, 384)
    self.start_logits, self.end_logits = x[0], x[1]

    # Fed into, Passed by the GELU layer
    self.start_logits = bert_gelu(self.start_logits)
    self.end_logits = bert_gelu(self.end_logits)

    # Fed into, Passed by the Softmax layer
    # Getting probability for 384 tokens
    self.start_logits = K.softmax(self.start_logits, axis=-1)
    self.end_logits = K.softmax(self.end_logits, axis=-1)

    return [self.start_logits, self.end_logits]

  def compute_output_shape(self, input_shape):
    # In Keras, when defining custom layer, 
    # the output dimension must be defined in compute_output_shape function
    return [(input_shape[0], self.seq_len), (input_shape[0], self.seq_len)]

In [2]:
# Load BERT model
from keras.layers import merge, dot, concatenate
from keras import metrics
import numpy as np

# Define a functoin loading BERT model
def get_bert_finetuning_model(model):
  # Input data is token-embedding and segment-embedding
  inputs = model.inputs[:2]

  # Output of Transformer : (batch_size, 384, 768)
  bert_transformer = model.layers[-1].output

  # Unmask the maksed tensors using NonMasking function
  x = NonMasking()(bert_transformer)

  # Show the start and end token of answer at last
  outputs_start, outputs_end = Start_End_Prediction(SEQ_LEN)(x)

  bert_model = keras.models.Model(inputs, [outputs_start, outputs_end])

  # Optimizer defined growing Learning_rate from 0 to 1.5e-5 gradually
  # Original RAdam changed to RAdamOptimizer for some issue
  optimizer_warmup = RAdamOptimizer(learning_rate = 1.5e-5, warmup_proportion=0.2, epsilon=1e-6, weight_decay = 0.01)

  # Final Model compile
  bert_model.compile(
      optimizer = optimizer_warmup,
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy']
  )

  return bert_model

In [31]:
# Display format customize in wrap for Google Colab (Youngsun)
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [29]:
# Code for loading pretrained BERT and fine_tuned trained model
bert_model = get_bert_finetuning_model(model)
path = "gdrive/MyDrive/Colab Notebooks/squad"
bert_model.load_weights(path+"/(Uncased)Squad.h5")

# 1. Baseline Testing (Without Re-training)

In [174]:
# # SQuAD Test Dataset Setup
# # Since test dataset is different in form with train dataset, (question, context format is same, but answer is different)
# # A function make format unification required to be made

# def squad_json_to_dataframe_dev(input_file_path, record_path = ['data', 'paragraphs', 'qas', 'answers'], verbos = 1):
#   """
#   input_file_path: path to the squad json file.
#   record_path: path to deepest level in json file default value is
#   ['data','paragraphs','qas','answers']
#   verbose: 0 to suppress it default is 1
#   """
#   if verbose:
#     print("Reading the json file")
#   file = json.loads(open(input_file_path).read())
#   if verbose:
#     print("processing...")
  
#   # Parsing different Level's in the json file
#   js = pd.io.json.json_normalize(file, record_path)
#   m = pd.io.json.json_normalize(file, record_path[:-1])
#   r = pd.io.json.json_normalize(file, record_path[:-2])

#   # Combining it into single dataframe
#   idx = np.repeat(r['context'].values, r.qas.str.len())
#   m['context'] = idx
#   main = m[['id', 'question', 'context', 'answers']].set_index('id').reset_index()
#   main['c_id'] = main['context'].factorize()[0]
#   if verbose:
#     print("shape of the dataframe is {}".format(main.shape))
#     print("Done")
#   return main

In [379]:
# # Test dataset loading
# input_file_path = 'dev-v1.1.json'
# record_path = ['data', 'paragraphs', 'qas', 'answers']
# verbose = 0
# dev = squad_json_to_dataframe_dev(input_file_path=input_file_path, record_path=record_path)

import pandas as pd
test_context = pd.read_excel('Test_QA_mod_without0.xlsx')

In [380]:
# Check if test dataset is loaded well
# ** Why answer needs to be in test dataset??? (Youngsun)
test_context

,Index,Paper,Category,Question,Abstract,Answer
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...
...,...,...,...,...,...,...
67,449,"Expression of the immediate early gene, c-fos,...",Radio Frequency,What is the signal frequency?,Aims: To study the effect of acute exposure to...,900 MHz
68,453,Bone morphogenetic protein expression in newbo...,Research Subject,What animal has been used?,Effects of nonthermal radiofrequency radiation...,rats
69,453,Bone morphogenetic protein expression in newbo...,Radio Frequency,What is the signal frequency?,Effects of nonthermal radiofrequency radiation...,9.4 GHz
70,455,IRIDIUM exposure increases c-fos expression in...,Research Subject,What animal has been used?,With the rapid development of wireless communi...,mice


In [381]:
# A function definition converting answers into list type for easy handling
# text_len : Number of answers
def get_text(text_len, answers):
  texts = []
  for i in range(text_len):
    texts.append(answers[i]['Answer'])
  return texts

In [382]:
test_context.head(5)

,Index,Paper,Category,Question,Abstract,Answer
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...


In [383]:
# # Making a column for the number of answers
# test_context['answer_len'] = test_context['Answer'].map(lambda x: x)

In [384]:
print(test_context['Answer'][0])
# print()
# print(test_context['Answer'][0][0]['text'])

Rats


In [385]:
# A function definition converting answers into list type for easy handling
# text_len : Number of answers
def get_text(answers):
  texts = []
  texts.append(answers)
  return texts

In [386]:
# Running the above function
# Taking the num of answers & answers as input, and converting them into List type
# get_text(1, test_context['Answer'][0])
get_text(test_context['Answer'][0])

['Rats']

In [387]:
# Doing the above job to all the test dataset
test_context['Answer_list'] = test_context.apply(lambda x: get_text(x['Answer']), axis=1)

In [388]:
test_context.head(5)

,Index,Paper,Category,Question,Abstract,Answer,Answer_list
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats,[Rats]
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz,[900 MHz]
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats,[rats]
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz,[900 MHz]
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...,[pulsed emission with a specific absorption ra...


In [389]:
test_context['Answer_list']

0                                                [Rats]
1                                             [900 MHz]
2                                                [rats]
3                                             [900 MHz]
4     [pulsed emission with a specific absorption ra...
                            ...                        
67                                            [900 MHz]
68                                               [rats]
69                                            [9.4 GHz]
70                                               [mice]
71                                            [1.6-GHz]
Name: Answer_list, Length: 72, dtype: object

In [390]:
# Making Input Data (Test Dataset) for BERT Model -- Embedding
# Utilizaing the above Input model in the past, but without answers!

# Column name of context
FINE_DATA_COLUMN = "Abstract"
# Column name of question
FINE_QUESTION_COLUMN = "Question"
# Column name of answer ('text')
FINE_TEXT = "Answer_list"

In [394]:
# Copied & Pasted, and a bit modified from the above function

# Define a function calculating the length of answer in context dataset all at once
def convert_data(data_df):
  global tokenizer
  indices, segments, target_start, target_end = [], [], [], []

  for i in tqdm(range(len(data_df))):
    # que : List of tokenized question
    que, _ = tokenizer.encode(data_df[FINE_QUESTION_COLUMN][i])
    # doc : List of tokenized context
    doc, _ = tokenizer.encode(data_df[FINE_DATA_COLUMN][i])

    # [CLS] token deleted in context
    doc.pop(0)

    # Length of question & context
    que_len = len(que)
    doc_len = len(doc)
    # 1. Length of question
    # The question is cut by the length of 64
    if que_len > 64:
      que = que[:63]
      que.append(102) # [SEP] token added to make it clear the question block
    # 2. Total length of question and context
    # The total input is cut by the length of 384
    if len(que+doc) > SEQ_LEN:
      while len(que+doc) != SEQ_LEN:
        doc.pop(-1)
      doc.pop(-1)
      doc.append(102)

    # Segment embedding
    # Question : 0 / Context 1 / Padding : 0 (remaining part for short sentences)
        
    ############################
    ###### Segment 예시 ########
    ############################
    
    # question, context, padding
    # 00000000, 1111111, 0000000
    
    segment = [0]*len(que) + [1]*len(doc) + [0]*(SEQ_LEN-len(que)-len(doc))

    # Padding
    if len(que+doc) <= SEQ_LEN:
      while len(que+doc) != SEQ_LEN:
        doc.append(0)

    # Final Input 'ids' (Question + Context)
    ids = que + doc

    # Sliding Part
    texts = data_df[FINE_TEXT][i]
    
    for text_element in texts:
      text = tokenizer.encode(text_element)[0]
      text_slide_len = len(text[1:-1]) # text_slide_len = 8

      for j in range(0, (len(doc))):
        # exist_flag : showing whether it is answerable or not (similar with is_unanswerable in SimpleTransforemr)
        # 0 : No answer / 1 : Having answer
        exist_flag = 0 
        if text[1:-1] == doc[j:j+text_slide_len]: # [0:8]->[1:9]->[2:10]->..->[159:160]
          # Assign the location of answer (start, end)
          ans_start = j + len(que)
          ans_end = j + text_slide_len - 1 + len(que)
          # If matched, exist_flag changed to 1
          exist_flag = 1
          break

      # When no answer case (exist_flag = 0), starting & ending value become SEQ_LEN
      # All the data of starting, ending = 384 (SEQ_LEN) will be deleted from the list
      if exist_flag == 0:
        ans_start = SEQ_LEN
        ans_end = SEQ_LEN

    # Input(ids), Segment saving into list type (indices, segments)
    indices.append(ids)
    segments.append(segment)
    # Starting and ending info saving into list type (target_start, target_end)
    target_start.append(ans_start)
    target_end.append(ans_end)

  # Converting the 4 lists into numpy array
  indices_x = np.array(indices)
  segments = np.array(segments)
  target_start = np.array(target_start)
  target_end = np.array(target_end)

  # The cut part saved in del_list and deleted from data
  del_list = np.where(target_start != SEQ_LEN)[0]
  not_del_list = np.where(target_start == SEQ_LEN)[0]
  indices_x = indices_x[del_list]
  segments = segments[del_list]
  
  target_start = target_start[del_list]
  target_end = target_end[del_list]

  return [indices_x, segments], del_list



# # Copied & Pasted, and a bit modified from the above function
# # Define a function calculating the length of answer in context dataset all at once
# def convert_data(test_context, test_question):
#   global tokenizer
#   indices, segments = [], []

#   que, _ = tokenizer.encode(test_question['Question'])
#   doc, _ = tokenizer.encode(test_context['Abstract'])
#   doc.pop(0)

#   que_len = len(que)
#   doc_len = len(doc)

#   # 1. Length of question
#   # The question is cut by the length of 64
#   if que_len > 64:
#     que = que[:63]
#     que.append(102) # [SEP] token added to make it clear the question block
  
#   # 2. Total length of question and context
#   # The total input is cut by the length of 384
#   if len(que+doc) > SEQ_LEN:
#     while len(que+doc) != SEQ_LEN:
#       doc.pop(-1)
#     doc.pop(-1)
#     doc.append(102)

#     # Segment embedding
#     # Question : 0 / Context 1 / Padding : 0 (remaining part for short sentences)
        
#     ############################
#     ###### Segment 예시 ########
#     ############################
    
#     # question, context, padding
#     # 00000000, 1111111, 0000000
    
#   segment = [0]*len(que) + [1]*len(doc) + [0]*(SEQ_LEN-len(que)-len(doc))

#   # Padding
#   if len(que+doc) <= SEQ_LEN:
#     while len(que+doc) != SEQ_LEN:
#       doc.append(0)

#   # Final Input 'ids' (Question + Context)
#   ids = que + doc

#   # Input(ids), Segment saving into list type (indices, segments)
#   indices.append(ids)
#   segments.append(segment)

#   # Converting the 4 lists into numpy array
#   indices = np.array(indices)
#   segments = np.array(segments)
  
#   # print(indices, segments)
#   return [indices, segments]

In [395]:
# Load of the Calculator Function
def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
  data_df[QUESTION_COLUMN] = data_df[QUESTION_COLUMN].astype(str)
  data_x, data_y, del_list = convert_data(data_df)

  return data_x, data_y, del_list

In [396]:
# Converting the test dataset into BERT input format
dev_bert_input = convert_data(test_context)

100%|██████████| 72/72 [00:00<00:00, 254.44it/s]


In [397]:
# Format : Token + Segment
dev_bert_input

([array([[  101,  2054,  4111, ...,     0,     0,     0],
         [  101,  2054,  2003, ...,     0,     0,     0],
         [  101,  2054,  4111, ..., 11721,  3676,   102],
         ...,
         [  101,  2054,  2003, ...,  1012,  1006,   102],
         [  101,  2054,  4111, ...,     0,     0,     0],
         [  101,  2054,  2003, ...,     0,     0,     0]]),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 1, 1],
         ...,
         [0, 0, 0, ..., 1, 1, 1],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])],
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71]))

In [398]:
# Cut the part over length of 384 ('del_list')
dev_bert_input, del_list = dev_bert_input[0], dev_bert_input[1]
dev = test_context.iloc[del_list]
dev = test_context.reset_index(drop=True)

In [399]:
dev

,Index,Paper,Category,Question,Abstract,Answer,Answer_list
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats,[Rats]
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz,[900 MHz]
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats,[rats]
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz,[900 MHz]
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...,[pulsed emission with a specific absorption ra...
...,...,...,...,...,...,...,...
67,449,"Expression of the immediate early gene, c-fos,...",Radio Frequency,What is the signal frequency?,Aims: To study the effect of acute exposure to...,900 MHz,[900 MHz]
68,453,Bone morphogenetic protein expression in newbo...,Research Subject,What animal has been used?,Effects of nonthermal radiofrequency radiation...,rats,[rats]
69,453,Bone morphogenetic protein expression in newbo...,Radio Frequency,What is the signal frequency?,Effects of nonthermal radiofrequency radiation...,9.4 GHz,[9.4 GHz]
70,455,IRIDIUM exposure increases c-fos expression in...,Research Subject,What animal has been used?,With the rapid development of wireless communi...,mice,[mice]


In [400]:
# Save the only tokenized sentences input (1~10527)
# Without the segment part
indexes = dev_bert_input[0]

In [401]:
indexes

array([[  101,  2054,  4111, ...,     0,     0,     0],
       [  101,  2054,  2003, ...,     0,     0,     0],
       [  101,  2054,  4111, ..., 11721,  3676,   102],
       ...,
       [  101,  2054,  2003, ...,  1012,  1006,   102],
       [  101,  2054,  4111, ...,     0,     0,     0],
       [  101,  2054,  2003, ...,     0,     0,     0]])

In [402]:
# BERT prediction start to all the test dataset (10,527 * 384 tokens * 787 features), taking about 2 mins
bert_predictions = bert_model.predict(dev_bert_input)

In [403]:
bert_predictions

[array([[2.1831365e-06, 5.2993890e-04, 3.5733028e-05, ..., 1.7509587e-06,
         1.7437283e-06, 1.7432195e-06],
        [2.6194368e-07, 3.2619182e-06, 2.1690602e-07, ..., 2.0423164e-07,
         2.0465181e-07, 2.0481698e-07],
        [9.2186965e-06, 3.5764650e-03, 2.6527347e-04, ..., 6.2507711e-06,
         6.2582508e-06, 6.1225614e-06],
        ...,
        [7.5015841e-08, 5.2319854e-07, 6.7134835e-08, ..., 6.9372497e-08,
         8.2671967e-08, 6.7741446e-08],
        [9.2734663e-06, 1.2762017e-03, 1.5390942e-04, ..., 6.4712258e-06,
         6.4667647e-06, 6.4754668e-06],
        [8.1038785e-08, 1.6732034e-07, 8.0639438e-08, ..., 8.0691137e-08,
         8.0757346e-08, 8.0760877e-08]], dtype=float32),
 array([[2.2052884e-06, 2.5964607e-04, 8.6874381e-05, ..., 2.0166447e-06,
         2.0076218e-06, 2.0068044e-06],
        [8.9909190e-07, 2.3353562e-06, 8.9983473e-07, ..., 8.5442639e-07,
         8.5832454e-07, 8.6056298e-07],
        [1.0944999e-05, 2.9103600e-03, 1.3444157e-03, ...,

In [404]:
# Among 384 tokens, selecting the one having the largest probability
start_indexes = np.argmax(bert_predictions[0], axis=-1)
end_indexes = np.argmax(bert_predictions[1], axis=-1)

In [405]:
# Delete some non-sense case 
# e.g. position of starting token (start_indexes) is bigger than the ending token (end_indexes)
not_del_list = np.where(start_indexes <= end_indexes)[0]

In [406]:
not_del_list.shape

(68,)

In [407]:
start_indexes = start_indexes[not_del_list]
end_indexes = end_indexes[not_del_list]
indexes = indexes[not_del_list]

In [408]:
# Positions of starting tokens
start_indexes[0:100]

array([108, 121, 100, 241, 282,  47,  59, 129, 125, 137, 160,  38, 203,
        43,  41,  77, 108, 101,  85, 101, 103, 232,  49, 101,  91,  31,
        99,  22,  63,  62,  73, 115, 101, 163,  81,  90, 108,  47, 164,
        68,  49,  80, 101,  98,  73, 294,  57,  68,  54,  68,  70,  97,
        63,  87,   8,  15,  19,  32,  46,  97,  66,  83,  57,  54,  82,
        92,  74,  77])

In [409]:
# Positions of end tokens
end_indexes[0:100]

array([108, 122, 100, 242, 282,  51,  59, 129, 125, 145, 166,  42, 205,
        47,  47,  79, 108, 105,  85, 106, 105, 233,  49, 101,  94,  34,
       108,  26,  69,  62,  74, 115, 103, 165,  86,  92, 110,  47, 164,
        71,  49,  83, 105,  98,  74, 298,  63,  69,  54,  68,  70, 103,
        63,  87,   8,  15,  19,  33,  48,  97,  68,  84,  57,  55,  82,
        95,  74,  81])

In [410]:
# After deleting the non-sense case, rearranging dataset
dev = dev.iloc[not_del_list].reset_index(drop=True)

In [411]:
dev

,Index,Paper,Category,Question,Abstract,Answer,Answer_list
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats,[Rats]
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz,[900 MHz]
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats,[rats]
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz,[900 MHz]
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...,[pulsed emission with a specific absorption ra...
...,...,...,...,...,...,...,...
63,449,"Expression of the immediate early gene, c-fos,...",Radio Frequency,What is the signal frequency?,Aims: To study the effect of acute exposure to...,900 MHz,[900 MHz]
64,453,Bone morphogenetic protein expression in newbo...,Research Subject,What animal has been used?,Effects of nonthermal radiofrequency radiation...,rats,[rats]
65,453,Bone morphogenetic protein expression in newbo...,Radio Frequency,What is the signal frequency?,Effects of nonthermal radiofrequency radiation...,9.4 GHz,[9.4 GHz]
66,455,IRIDIUM exposure increases c-fos expression in...,Research Subject,What animal has been used?,With the rapid development of wireless communi...,mice,[mice]


In [412]:
# Getting the length of test data
length = len(dev)

In [413]:
length

68

In [416]:
sentences = []
untokenized = []

for j in range(len(start_indexes)):
  sentence = []
  # Saving each tokenized word into the list of sentence (sentence = [])
  for i in range(start_indexes[j], end_indexes[j]+1):
    token_based_word = reverse_token_dict[indexes[j][i]]
    sentence.append(token_based_word)
  sentence_string = ""

  for w in sentence:
    # Special token ## delete, if a token starts with ##
    if w.startswith("##"):
      w = w.replace("##", "")
    # If a token has no ##, a space added to the word
    else:
      w = " " + w
    # Putting together all the tokens in list format
    sentence_string += w
  
  # If senetence_string starts with a space (" "), delete the space
  if sentence_string.startswith(" "):
    sentence_string = "" + sentence_string[1:]
  
  # After putting together all the list of tokens, and assigning it in the list of the 'Untokenized'
  untokenized.append(sentence_string)
  sentences.append(sentence)

In [417]:
print(sentences[:30])
print('\n')
print(untokenized[:30])

[['rats'], ['900', 'mhz'], ['rats'], ['900', 'mhz'], ['32'], ['bal', '##b', '/', 'c', 'mice'], ['8'], ['rats'], ['96'], ['900', 'mhz', ',', '1800', 'mhz', 'and', '245', '##0', 'mhz'], ['5', '.', '84', '×', '10', '-', '4'], ['gs', '##m', '-', '1800', 'mhz'], ['3', '.', '22'], ['wi', '##star', 'al', '##bino', 'rats'], ['24', 'male', 'wi', '##star', 'al', '##bino', 'rats'], ['210', '##0', 'mhz'], ['mice'], ['c', '##57', '##bl', '/', '6'], ['mice'], ['4', '.', '0', 'w', '/', 'kg'], ['4', '.', '0'], ['sham', 'rats'], ['mice'], ['twelve'], ['900', 'and', '1800', 'mhz'], ['1800', '##m', '##h', '##z'], ['rats', 'were', 'divided', 'into', 'the', 'following', 'groups', ':', 'sham', 'rats'], ['900', 'and', '210', '##0', 'mhz'], ['sp', '##rag', '##ue', '-', 'da', '##wley', 'rats'], ['20']]


['rats', '900 mhz', 'rats', '900 mhz', '32', 'balb / c mice', '8', 'rats', '96', '900 mhz , 1800 mhz and 2450 mhz', '5 . 84 × 10 - 4', 'gsm - 1800 mhz', '3 . 22', 'wistar albino rats', '24 male wistar albino r

In [422]:
# Converting the answers into list format
dev_answers = []
for i in range(length):
  dev_answer = []
  texts_dict = test_context['Answer']

  for j in range(len(texts_dict)):
    dev_answer.append(texts_dict[j])
  dev_answers.append(dev_answer)

In [436]:
dev_answers[0]

['rats',
 '900 mhz',
 'rats',
 '900 mhz',
 'pulsed emission with a specific absorption rate ( sar ) of 4 w / kg and continuous emission with high sar ( 32 w / kg )',
 'balb / c mice',
 'forty - eight',
 'wistar rats',
 '96',
 '900 mhz , 1800 mhz and 2450 mhz',
 '5 . 84 × 10 - 4 w / kg , 5 . 94 × 10 - 4 w / kg and 6 . 4 × 10 - 4 w / kg',
 'rats',
 '1800 mhz',
 '3 . 22 w / kg',
 'wistar albino rats',
 '24',
 '2100 mhz',
 'mice',
 'whole - body exposed ( nexp = 8 ) for 2 hr to gsm 1800 mhz mobile phone radiation at an average electric field intensity range of 4 . 3 - 17 . 5 v / m or sham - exposed ( nse = 8 )',
 '1800 mhz',
 'c57bl / 6 mice',
 '835 mhz',
 '4 . 0 w / kg',
 'rats',
 '900 mhz',
 'balb / c mice',
 'twelve',
 '900 and 1800 mhz',
 'rats',
 '1800mhz',
 'rats',
 '900 and 2100 mhz',
 'sprague - dawley rats',
 '20',
 '900 mhz',
 'mice',
 '835 mhz',
 '4 . 0 w / kg',
 'c57bl / 6 mice',
 '835 mhz',
 '4 . 0 w / kg',
 'chicken eggs',
 '450 eggs per treatment in three successive rounds p

In [426]:
# Tokenize the answers
dev_tokens = []
for i in dev_answers:
  dev_tokened = []
  for j in i:
    temp_token = tokenizer.tokenize(j)
    # Tokenize an answer
    temp_token.pop(0)
    # [CLS] elimination
    temp_token.pop(-1)
    # [SEP] elimination
    dev_tokened.append(temp_token)
  dev_tokens.append(dev_tokened)

In [427]:
print(dev_tokens[:5])

[[['rats'], ['900', 'mhz'], ['rats'], ['900', 'mhz'], ['pulsed', 'emission', 'with', 'a', 'specific', 'absorption', 'rate', '(', 'sar', ')', 'of', '4', 'w', '/', 'kg', 'and', 'continuous', 'emission', 'with', 'high', 'sar', '(', '32', 'w', '/', 'kg', ')'], ['bal', '##b', '/', 'c', 'mice'], ['forty', '-', 'eight'], ['wi', '##star', 'rats'], ['96'], ['900', 'mhz', ',', '1800', 'mhz', 'and', '245', '##0', 'mhz'], ['5', '.', '84', '×', '10', '-', '4', 'w', '/', 'kg', ',', '5', '.', '94', '×', '10', '-', '4', 'w', '/', 'kg', 'and', '6', '.', '4', '×', '10', '-', '4', 'w', '/', 'kg'], ['rats'], ['1800', 'mhz'], ['3', '.', '22', 'w', '/', 'kg'], ['wi', '##star', 'al', '##bino', 'rats'], ['24'], ['210', '##0', 'mhz'], ['mice'], ['whole', '-', 'body', 'exposed', '(', 'ne', '##x', '##p', '=', '8', ')', 'for', '2', 'hr', 'to', 'gs', '##m', '1800', 'mhz', 'mobile', 'phone', 'radiation', 'at', 'an', 'average', 'electric', 'field', 'intensity', 'range', 'of', '4', '.', '3', '-', '17', '.', '5', 'v',

In [428]:
# Converting tokenized answers into sentences
# And putting all together
dev_answer_lists = []
for dev_answers in dev_tokens:
  dev_answer_list = []
  for dev_answer in dev_answers:
    dev_answer_string = " ".join(dev_answer)
    dev_answer_list.append(dev_answer_string)
  dev_answer_lists.append(dev_answer_list)

In [429]:
print(dev_answer_lists[:5])

[['rats', '900 mhz', 'rats', '900 mhz', 'pulsed emission with a specific absorption rate ( sar ) of 4 w / kg and continuous emission with high sar ( 32 w / kg )', 'bal ##b / c mice', 'forty - eight', 'wi ##star rats', '96', '900 mhz , 1800 mhz and 245 ##0 mhz', '5 . 84 × 10 - 4 w / kg , 5 . 94 × 10 - 4 w / kg and 6 . 4 × 10 - 4 w / kg', 'rats', '1800 mhz', '3 . 22 w / kg', 'wi ##star al ##bino rats', '24', '210 ##0 mhz', 'mice', 'whole - body exposed ( ne ##x ##p = 8 ) for 2 hr to gs ##m 1800 mhz mobile phone radiation at an average electric field intensity range of 4 . 3 - 17 . 5 v / m or sham - exposed ( ns ##e = 8 )', '1800 mhz', 'c ##57 ##bl / 6 mice', '83 ##5 mhz', '4 . 0 w / kg', 'rats', '900 mhz', 'bal ##b / c mice', 'twelve', '900 and 1800 mhz', 'rats', '1800 ##m ##h ##z', 'rats', '900 and 210 ##0 mhz', 'sp ##rag ##ue - da ##wley rats', '20', '900 mhz', 'mice', '83 ##5 mhz', '4 . 0 w / kg', 'c ##57 ##bl / 6 mice', '83 ##5 mhz', '4 . 0 w / kg', 'chicken eggs', '450 eggs per trea

In [430]:
# Untokenizing (including deleting " ##" etc)
dev_strings_end = []
for dev_strings in dev_answer_lists:
  dev_strings_processed = []
  for dev_string in dev_strings:
    dev_string = dev_string.replace(" ##", "")
    dev_strings_processed.append(dev_string)
  dev_strings_end.append(dev_strings_processed)

In [431]:
dev_answers = dev_strings_end

In [432]:
print(dev_answers[:5])

[['rats', '900 mhz', 'rats', '900 mhz', 'pulsed emission with a specific absorption rate ( sar ) of 4 w / kg and continuous emission with high sar ( 32 w / kg )', 'balb / c mice', 'forty - eight', 'wistar rats', '96', '900 mhz , 1800 mhz and 2450 mhz', '5 . 84 × 10 - 4 w / kg , 5 . 94 × 10 - 4 w / kg and 6 . 4 × 10 - 4 w / kg', 'rats', '1800 mhz', '3 . 22 w / kg', 'wistar albino rats', '24', '2100 mhz', 'mice', 'whole - body exposed ( nexp = 8 ) for 2 hr to gsm 1800 mhz mobile phone radiation at an average electric field intensity range of 4 . 3 - 17 . 5 v / m or sham - exposed ( nse = 8 )', '1800 mhz', 'c57bl / 6 mice', '835 mhz', '4 . 0 w / kg', 'rats', '900 mhz', 'balb / c mice', 'twelve', '900 and 1800 mhz', 'rats', '1800mhz', 'rats', '900 and 2100 mhz', 'sprague - dawley rats', '20', '900 mhz', 'mice', '835 mhz', '4 . 0 w / kg', 'c57bl / 6 mice', '835 mhz', '4 . 0 w / kg', 'chicken eggs', '450 eggs per treatment in three successive rounds per treatment', 'gsm ( 1 . 8 ghz ) , dect 

# 1. Baseline Evaluation - F1 & EM

In [433]:
# F1 & EM function defined by myself
from collections import Counter
import string

# In normalize step, doing some works such as converting words into lower case,
# and deleting punctuations, clearing unnecessary spaces etc
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)
  
  def white_space_fix(text):
    return ' '.join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()
  
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def exact_match_score(prediction, ground_truth):
  return (normalize_answer(prediction) == normalize_answer(ground_truth))

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
  scores_for_ground_truths = []
  for ground_truth in ground_truths:
    score = metric_fn(prediction, ground_truth)
    scores_for_ground_truths.append(score)
  return max(scores_for_ground_truths)

In [434]:
# Calculating F1 Score
# Theoretical Max : 88%
f1_sum = 0

for i in range(len(untokenized)):
  f1 = metric_max_over_ground_truths(f1_score, untokenized[i], dev_answers[i])
  f1_sum += f1
print("f1 score: ", f1_sum/length)

f1 score:  0.8190902464122276


In [435]:
# Calculating EM score
# Theoretical Max : 80%
EM_sum = 0

for i in range(len(untokenized)):
  EM = metric_max_over_ground_truths(exact_match_score, untokenized[i], dev_answers[i])
  EM_sum += EM
print("EM score: ", EM_sum/length)

EM score:  0.6323529411764706


# 2. Fine-tuning Training (Re-training)

In [437]:
# Our dataset loading in Pandas format
import pandas as pd
fine_train = pd.read_excel('Train_QA_mod.xlsx')

In [438]:
fine_train

,Index,Paper,Category,Question,Abstract,Answer
0,1,Experimental Studies of Brain Tumor Developmen...,Research Subject,What animal has been used?,It has been suggested that electromagnetic fie...,Fischer 344 rats
1,1,Experimental Studies of Brain Tumor Developmen...,Number of Research Subject,How many animals were used?,It has been suggested that electromagnetic fie...,37 experimental rats and 37 matched controls
2,1,Experimental Studies of Brain Tumor Developmen...,Radio Frequency,What is the signal frequency?,It has been suggested that electromagnetic fie...,915 MHz
3,1,Experimental Studies of Brain Tumor Developmen...,Other Units of Exposure Level,How much W/kg was used?,It has been suggested that electromagnetic fie...,NaN
4,3,Chronic Exposure of Cancer-Prone Mice to Low-L...,Research Subject,What animal has been used?,The purpose of this study was to determine whe...,C3H/ HeJ mice
...,...,...,...,...,...,...
663,368,Melatonin modulates 900 Mhz microwave-induced ...,Other Units of Exposure Level,How much W/kg was used?,Microwaves (MW) from cellular phones may affec...,NaN
664,370,Does 900 MHz GSM Mobile Phone Exposure Affect ...,Research Subject,What animal has been used?,This study investigated the effects of cell ph...,Sprague-Dawley rats
665,370,Does 900 MHz GSM Mobile Phone Exposure Affect ...,Number of Research Subject,How many animals were used?,This study investigated the effects of cell ph...,Sixteen
666,370,Does 900 MHz GSM Mobile Phone Exposure Affect ...,Radio Frequency,What is the signal frequency?,This study investigated the effects of cell ph...,NaN


In [ ]:
# # Hyperparameters set up

# # Input sequence maximum length setup (Question + Context)
# SEQ_LEN = 384

# # Batch size (training size at one time)
# # Refer to this site https://mccormickml.com/2019/07/22/BERT-fine-tuning/
# BATCH_SIZE = 32

# EPOCH = 4

# # Learning rate is set in very small number like 1.5e-5.
# # In fine tuning session, Learning rate must be very small 
# # (Optimizer RAdam starts from LR 0 to 1.5e-5)
# LR = 1.5e-5

# # Model location designation
# pretrained_path = "bert"
# # Pretrained hyperparameters like weights
# checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
# # Word corpus file location designation
# vocab_path =  os.path.join(pretrained_path, 'vocab.txt')

# # Configuration file designation
# config_path = os.path.join(pretrained_path, 'bert_config.json')

# # Column name of context
# FINE_DATA_COLUMN = "Abstract"
# # Column name of question
# FINE_QUESTION_COLUMN = "Question"
# # Column name of answer ('text')
# FINE_TEXT = "Answer"

In [439]:
# Hyperparameters set up

# Input sequence maximum length setup (Question + Context)
SEQ_LEN = 384

# Batch size (training size at one time)
# Refer to this site https://mccormickml.com/2019/07/22/BERT-fine-tuning/
BATCH_SIZE = 32

EPOCH = 4

# Learning rate is set in very small number like 1.5e-5.
# In fine tuning session, Learning rate must be very small 
# (Optimizer RAdam starts from LR 0 to 1.5e-5)
LR = 1.5e-5

# Model location designation
pretrained_path = "bert"
# Pretrained hyperparameters like weights
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
# Word corpus file location designation
vocab_path =  os.path.join(pretrained_path, 'vocab.txt')

# Configuration file designation
config_path = os.path.join(pretrained_path, 'bert_config.json')

# Column name of context
FINE_DATA_COLUMN = "Abstract"
# Column name of question
FINE_QUESTION_COLUMN = "Question"
# Column name of answer ('text')
FINE_TEXT = "Answer"

In [440]:
# Dictionary Construction for indexing each word
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
  for line in reader:
    token = line.strip()
    token_dict[token] = len(token_dict)

In [441]:
token_dict

{'[PAD]': 0,
 '[unused0]': 1,
 '[unused1]': 2,
 '[unused2]': 3,
 '[unused3]': 4,
 '[unused4]': 5,
 '[unused5]': 6,
 '[unused6]': 7,
 '[unused7]': 8,
 '[unused8]': 9,
 '[unused9]': 10,
 '[unused10]': 11,
 '[unused11]': 12,
 '[unused12]': 13,
 '[unused13]': 14,
 '[unused14]': 15,
 '[unused15]': 16,
 '[unused16]': 17,
 '[unused17]': 18,
 '[unused18]': 19,
 '[unused19]': 20,
 '[unused20]': 21,
 '[unused21]': 22,
 '[unused22]': 23,
 '[unused23]': 24,
 '[unused24]': 25,
 '[unused25]': 26,
 '[unused26]': 27,
 '[unused27]': 28,
 '[unused28]': 29,
 '[unused29]': 30,
 '[unused30]': 31,
 '[unused31]': 32,
 '[unused32]': 33,
 '[unused33]': 34,
 '[unused34]': 35,
 '[unused35]': 36,
 '[unused36]': 37,
 '[unused37]': 38,
 '[unused38]': 39,
 '[unused39]': 40,
 '[unused40]': 41,
 '[unused41]': 42,
 '[unused42]': 43,
 '[unused43]': 44,
 '[unused44]': 45,
 '[unused45]': 46,
 '[unused46]': 47,
 '[unused47]': 48,
 '[unused48]': 49,
 '[unused49]': 50,
 '[unused50]': 51,
 '[unused51]': 52,
 '[unused52]': 53,

In [442]:
# Tokenizer Construction
tokenizer = Tokenizer(token_dict)

In [443]:
# Checking if the tokenizer works well
print(tokenizer.tokenize("Youngsun is really awesome."))
print(tokenizer.tokenize("We can manipulate AI."))

['[CLS]', 'young', '##sun', 'is', 'really', 'awesome', '.', '[SEP]']
['[CLS]', 'we', 'can', 'manipulate', 'ai', '.', '[SEP]']


In [444]:
# For later use, updated dictionary constructed (number - word order)
reverse_token_dict = {v : k for k, v in token_dict.items()}

In [445]:
reverse_token_dict

{0: '[PAD]',
 1: '[unused0]',
 2: '[unused1]',
 3: '[unused2]',
 4: '[unused3]',
 5: '[unused4]',
 6: '[unused5]',
 7: '[unused6]',
 8: '[unused7]',
 9: '[unused8]',
 10: '[unused9]',
 11: '[unused10]',
 12: '[unused11]',
 13: '[unused12]',
 14: '[unused13]',
 15: '[unused14]',
 16: '[unused15]',
 17: '[unused16]',
 18: '[unused17]',
 19: '[unused18]',
 20: '[unused19]',
 21: '[unused20]',
 22: '[unused21]',
 23: '[unused22]',
 24: '[unused23]',
 25: '[unused24]',
 26: '[unused25]',
 27: '[unused26]',
 28: '[unused27]',
 29: '[unused28]',
 30: '[unused29]',
 31: '[unused30]',
 32: '[unused31]',
 33: '[unused32]',
 34: '[unused33]',
 35: '[unused34]',
 36: '[unused35]',
 37: '[unused36]',
 38: '[unused37]',
 39: '[unused38]',
 40: '[unused39]',
 41: '[unused40]',
 42: '[unused41]',
 43: '[unused42]',
 44: '[unused43]',
 45: '[unused44]',
 46: '[unused45]',
 47: '[unused46]',
 48: '[unused47]',
 49: '[unused48]',
 50: '[unused49]',
 51: '[unused50]',
 52: '[unused51]',
 53: '[unused52]',

In [446]:
# Checking if the tokenizer works well again in QnA format ([CLS} Question [SEP] Context [SEP] order)
print(tokenizer.tokenize("Where does Youngsun live in now?", "My name is Youngsun Jang, and I live in US now."))

['[CLS]', 'where', 'does', 'young', '##sun', 'live', 'in', 'now', '?', '[SEP]', 'my', 'name', 'is', 'young', '##sun', 'jang', ',', 'and', 'i', 'live', 'in', 'us', 'now', '.', '[SEP]']


In [447]:
# Word Embedding (Numberical representation)
print(tokenizer.encode("Where does Youngsun live in now?", "My name is Youngsun Jang, and I live in US now."))

([101, 2073, 2515, 2402, 19729, 2444, 1999, 2085, 1029, 102, 2026, 2171, 2003, 2402, 19729, 23769, 1010, 1998, 1045, 2444, 1999, 2149, 2085, 1012, 102], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [448]:
fine_train.head(5)

,Index,Paper,Category,Question,Abstract,Answer
0,1,Experimental Studies of Brain Tumor Developmen...,Research Subject,What animal has been used?,It has been suggested that electromagnetic fie...,Fischer 344 rats
1,1,Experimental Studies of Brain Tumor Developmen...,Number of Research Subject,How many animals were used?,It has been suggested that electromagnetic fie...,37 experimental rats and 37 matched controls
2,1,Experimental Studies of Brain Tumor Developmen...,Radio Frequency,What is the signal frequency?,It has been suggested that electromagnetic fie...,915 MHz
3,1,Experimental Studies of Brain Tumor Developmen...,Other Units of Exposure Level,How much W/kg was used?,It has been suggested that electromagnetic fie...,NaN
4,3,Chronic Exposure of Cancer-Prone Mice to Low-L...,Research Subject,What animal has been used?,The purpose of this study was to determine whe...,C3H/ HeJ mice


In [449]:
print("Question: ", fine_train.loc[0, 'Question'])
print("Context: ", fine_train.loc[0, 'Abstract'])
print("Answer: ", fine_train.loc[0, 'Answer'])

Question:  What animal has been used?
Context:  It has been suggested that electromagnetic fields (EMFs) act as a promoter late in the carcinogenesis process. To date, however, noconvincing laboratory evidence has been obtained indicating that EMFs cause tumour promotion at non-thermal exposure levels. The effects of EMF exposure in a rat brain glioma model were investigated. The exposure consisted of 915 MHz microwaves, both as continuous waves (1 W), and modulated with 4, 8, 16 and 200 Hz in 0.5 ms pulses and 50 Hz in 6 ms pulses (2 W per pulse). Fischer 344 rats of both sexes, weighing 150–250 g, were used in the experiments. 5000 RG2 cells in 5 μ1 nutrient solution were injected by the stereotaxic technique into the head of the right caudate nucleus in 37 experimental rats and 37 matched controls. The exposed animals were kept unanaesthetized in well ventilated transverse electromagnetic (TEM) cells producing 915 MHz continuous or modulated microwaves. Exposure was started on day f

In [450]:
# QA is to confirm the position of the Answer token in Context 
# position of Saint Bernadette Soubirous !
# That is, from 3002 - 2271

print("Question: ", tokenizer.tokenize(fine_train.loc[0, 'Question']))
print("Context: ", tokenizer.tokenize(fine_train.loc[0, 'Abstract']))
print("Answer: ", tokenizer.tokenize(fine_train.loc[0, 'Answer']))

print("Question: ", tokenizer.encode(fine_train.loc[0, 'Question']))
print("Context: ", tokenizer.encode(fine_train.loc[0, 'Abstract']))
print("Answer: ", tokenizer.encode(fine_train.loc[0, 'Answer']))

Question:  ['[CLS]', 'what', 'animal', 'has', 'been', 'used', '?', '[SEP]']
Context:  ['[CLS]', 'it', 'has', 'been', 'suggested', 'that', 'electromagnetic', 'fields', '(', 'em', '##fs', ')', 'act', 'as', 'a', 'promoter', 'late', 'in', 'the', 'car', '##cino', '##genesis', 'process', '.', 'to', 'date', ',', 'however', ',', 'no', '##con', '##vin', '##cing', 'laboratory', 'evidence', 'has', 'been', 'obtained', 'indicating', 'that', 'em', '##fs', 'cause', 'tu', '##mour', 'promotion', 'at', 'non', '-', 'thermal', 'exposure', 'levels', '.', 'the', 'effects', 'of', 'em', '##f', 'exposure', 'in', 'a', 'rat', 'brain', 'g', '##lio', '##ma', 'model', 'were', 'investigated', '.', 'the', 'exposure', 'consisted', 'of', '91', '##5', 'mhz', 'microwave', '##s', ',', 'both', 'as', 'continuous', 'waves', '(', '1', 'w', ')', ',', 'and', 'mod', '##ulated', 'with', '4', ',', '8', ',', '16', 'and', '200', 'hz', 'in', '0', '.', '5', 'ms', 'pulses', 'and', '50', 'hz', 'in', '6', 'ms', 'pulses', '(', '2', 'w', '

In [451]:
# Getting only token part excepting the segment part
fine_context = tokenizer.encode(fine_train.loc[0, 'Abstract'])[0]
fine_text = tokenizer.encode(fine_train.loc[0, 'Answer'])[0]

In [452]:
# CLS, SEP token delete
fine_text.pop(0)
fine_text.pop(-1)
print(fine_text)
print(len(fine_text))
len(fine_context)

[13042, 29386, 11432]
3


367

In [453]:
# To calculate the length of text (answer)
# Method : Doing 'Sliding' the context with the length of the answer
fine_text_slide_len = len(fine_text) # text_slide_len = 8

for j in range(0, (len(fine_context))):
  # exist_flag : showing whether it is answerable or not (similar with is_unanswerable in SimpleTransforemr)
  # 0 : No answer / 1 : Having answer
  exist_flag = 0 
  if fine_text == fine_context[j:j+fine_text_slide_len]: # [0:8]->[1:9]->[2:10]->..->[159:160]
    # Assign the location of answer (start, end)
    fine_ans_start = j
    fine_ans_end = j + fine_text_slide_len - 1
    # If matched, exist_flag changed to 1
    exist_flag = 1
    break

print("ans_start : {}, ans_end : {}".format(fine_ans_start, fine_ans_end))


ans_start : 121, ans_end : 123


In [454]:
# Check
# context[ans_start:ans_end]
print(fine_context[fine_ans_start:fine_ans_end+1], fine_text)

[13042, 29386, 11432] [13042, 29386, 11432]


In [455]:
# Define a function calculating the length of answer in context dataset all at once
def fine_convert_data(data_df):
  global tokenizer
  indices, segments, target_start, target_end = [], [], [], []

  for i in tqdm(range(len(data_df))):
    # que : List of tokenized question
    que, _ = tokenizer.encode(data_df[FINE_QUESTION_COLUMN][i])
    # doc : List of tokenized context
    doc, _ = tokenizer.encode(data_df[FINE_DATA_COLUMN][i])

    # [CLS] token deleted in context
    doc.pop(0)

    # Length of question & context
    que_len = len(que)
    doc_len = len(doc)
    # 1. Length of question
    # The question is cut by the length of 64
    if que_len > 64:
      que = que[:63]
      que.append(102) # [SEP] token added to make it clear the question block
    # 2. Total length of question and context
    # The total input is cut by the length of 384
    if len(que+doc) > SEQ_LEN:
      while len(que+doc) != SEQ_LEN:
        doc.pop(-1)
      doc.pop(-1)
      doc.append(102)

    # Segment embedding
    # Question : 0 / Context 1 / Padding : 0 (remaining part for short sentences)
        
    ############################
    ###### Segment 예시 ########
    ############################
    
    # question, context, padding
    # 00000000, 1111111, 0000000
    
    segment = [0]*len(que) + [1]*len(doc) + [0]*(SEQ_LEN-len(que)-len(doc))

    # Padding
    if len(que+doc) <= SEQ_LEN:
      while len(que+doc) != SEQ_LEN:
        doc.append(0)

    # Final Input 'ids' (Question + Context)
    ids = que + doc

    # Sliding Part
    fine_text = tokenizer.encode(data_df[FINE_TEXT].astype(str)[i])[0]
    fine_text_slide_len = len(fine_text[1:-1]) # text_slide_len = 8

    for j in range(0, (len(doc))):
      # exist_flag : showing whether it is answerable or not (similar with is_unanswerable in SimpleTransforemr)
      # 0 : No answer / 1 : Having answer
      exist_flag = 0 
      if fine_text[1:-1] == doc[j:j+fine_text_slide_len]: # [0:8]->[1:9]->[2:10]->..->[159:160]
        # Assign the location of answer (start, end)
        fine_ans_start = j + len(que)
        fine_ans_end = j + fine_text_slide_len - 1 + len(que)
        # If matched, exist_flag changed to 1
        exist_flag = 1
        break

    # When no answer case (exist_flag = 0), starting & ending value become SEQ_LEN
    # All the data of starting, ending = 384 (SEQ_LEN) will be deleted from the list
    if exist_flag == 0:
      fine_ans_start = SEQ_LEN
      fine_ans_end = SEQ_LEN

    # Input(ids), Segment saving into list type (indices, segments)
    indices.append(ids)
    segments.append(segment)
    # Starting and ending info saving into list type (target_start, target_end)
    target_start.append(fine_ans_start)
    target_end.append(fine_ans_end)

  # Converting the 4 lists into numpy array
  indices_x = np.array(indices)
  segments = np.array(segments)
  target_start = np.array(target_start)
  target_end = np.array(target_end)

  # The cut part saved in del_list and deleted from data
  del_list = np.where(target_start != SEQ_LEN)[0]
  not_del_list = np.where(target_start == SEQ_LEN)[0]
  indices_x = indices_x[del_list]
  segments = segments[del_list]
  target_start = target_start[del_list]
  target_end = target_end[del_list]

  return [indices_x, segments], [target_start, target_end], not_del_list

In [456]:
# Load of the Calculator Function
def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df[FINE_DATA_COLUMN] = data_df[FINE_DATA_COLUMN].astype(str)
  data_df[FINE_QUESTION_COLUMN] = data_df[FINE_QUESTION_COLUMN].astype(str)
  data_x, data_y, del_list = fine_convert_data(data_df)
  
  return data_x, data_y, del_list

In [457]:
# Train Data Coneversion Starting
fine_train_x, fine_train_y, z = load_data(fine_train)

100%|██████████| 668/668 [00:02<00:00, 249.32it/s]


In [458]:
print(fine_train_x[0].shape, fine_train_y[0].shape)

(451, 384) (451,)


In [459]:
# For Transfer Learning, Customized Layer needs to be added after 12 Encoder
# by defining 'Non-masking' function, BERT Model's masked tensors disclosed

class NonMasking(Layer):
  def __init__(self, **kwargs):
    self.supports_masking = True
    super(NonMasking, self).__init__(**kwargs)

  def build(self, input_shape):
    input_shape = input_shape
  
  def compute_mask(self, input, input_mask = None):
    return None

  def call(self, x, mask = None):
    return x

  def get_output_shape_for(self, input_shape):
    return input_shape

In [460]:
# Activation function definition
def bert_gelu(x):
  """Gaussian Error Linear Unit.
  This is a smoother version of the RELU.
  Original paper: https://arxiv.org/abs/1606.08415
  Args:
    x : float Tensor to perform activation
  Returns:
    'x' with the GELU activation applied.
  """
  cdf = 0.5*(1.0+ K.tanh(
      (np.sqrt(2/np.pi)*(x+0.044715 * K.pow(x,3)))))
  return x*cdf

In [461]:
# Activation Function Layer attached to Transformer
class Start_End_Prediction(Layer):
  def __init__(self, seq_len, **kwargs):
    self.seq_len = SEQ_LEN
    self.supports_masking = True
    super(Start_End_Prediction, self).__init__(**kwargs)

  def build(self, input_shape):
    # A tensor ('self.W') multiplied with the final layer (12 encoder, batch_size, 384, 768)
    # Making Output tensor as (384, 2 dimension (768->2))
    self.W = self.add_weight(name='kernel',
                             shape = (input_shape[2],2),
                             initializer = 'uniform',
                             trainable = True)
    super(Start_End_Prediction, self).build(input_shape)

  def call(self, x):
    # Redifine Output dimension as (384 * 768)
    x = K.reshape(x, shape=(-1, self.seq_len, K.shape(x)[2]))
    # Dot production between self.W and x
    # (batch_size, 384, 768) * (384, 2) = (batch_size, 384, 2)
    x = K.dot(x, self.W)
    # (batch_size, 384, 2) -> (2, batch_size, 384)
    x = K.permute_dimensions(x, (2, 0, 1))

    # Split the (2, batch_size, 384) into 2 (batch_size, 384)  --> start_logits & end_logits
    # start_logits = (batch_size, 384)
    # end_logits = (batch_size, 384)
    self.start_logits, self.end_logits = x[0], x[1]

    # Fed into, Passed by the GELU layer
    self.start_logits = bert_gelu(self.start_logits)
    self.end_logits = bert_gelu(self.end_logits)

    # Fed into, Passed by the Softmax layer
    # Getting probability for 384 tokens
    self.start_logits = K.softmax(self.start_logits, axis=-1)
    self.end_logits = K.softmax(self.end_logits, axis=-1)

    return [self.start_logits, self.end_logits]

  def compute_output_shape(self, input_shape):
    # In Keras, when defining custom layer, 
    # the output dimension must be defined in compute_output_shape function
    return [(input_shape[0], self.seq_len), (input_shape[0], self.seq_len)]

In [462]:
# Training -- Error
# sess = K.get_session()
# uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
# init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
# sess.run(init)

# bert_model = get_bert_finetuning_model(bert_model)
bert_model.summary()

# Training Start!
history = bert_model.fit(fine_train_x, fine_train_y, batch_size=5, shuffle=True, verbose=1, epochs=5)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 384)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 384)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 384, 768),  23440896    ['Input-Token[0][0]']            
 g)                              (30522, 768)]                                                    
                                                                                                  
 Embedding-Segment (Embedding)  (None, 384, 768)     1536        ['Input-Segment[0][0]']    

In [463]:
# It shows accuracy about 76% in the above
# Now need to get f1 score as alternative

from sklearn.metrics import classification_report
fine_preds = bert_model.predict(fine_train_x)

In [464]:
fine_start_indexes = np.argmax(fine_preds[0], axis=-1)
fine_end_indexes = np.argmax(fine_preds[1], axis=-1)

In [465]:
# Printing F1 score of two tokens (start, end)

# Start token ('start_indexes') : 82%
print(classification_report(fine_train_y[0], fine_start_indexes))
# End token ('end_indexes') : 85%
print(classification_report(fine_train_y[1], fine_end_indexes))

              precision    recall  f1-score   support

           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         3
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         4
          16       1.00      1.00      1.00         5
          17       1.00      1.00      1.00         1
          18       1.00      1.00      1.00         1
          19       1.00      1.00      1.00         5
          20       1.00      1.00      1.00         2
          21       1.00      1.00      1.00         6
          22       1.00      1.00      1.00         2
          24       1.00      1.00      1.00         1
          25       1.00      1.00      1.00         2
          26       1.00      1.00      1.00         1
          27       1.00      1.00      1.00         3
          28       1.00    

In [ ]:
# # Fine_tuned trained model save
# # Since it takes too much time, trained model saved in the 'gdrive'
# path = "gdrive/MyDrive/Colab Notebooks/squad"
# bert_model.save_weights(path+"/(Uncased)Squad.h5")

In [ ]:
# # Code for loading pretrained BERT and fine_tuned trained model
# bert_model = get_bert_finetuning_model(model)
# path = "gdrive/MyDrive/Colab Notebooks/squad"
# bert_model.load_weights(path+"/(Uncased)Squad.h5")

# 2. Fine-tuning Testing

In [467]:
# # Test dataset loading
import pandas as pd
test_context = pd.read_excel('Test_QA_mod_without0.xlsx')

In [468]:
# Check if test dataset is loaded well
# ** Why answer needs to be in test dataset??? (Youngsun)
test_context

,Index,Paper,Category,Question,Abstract,Answer
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...
...,...,...,...,...,...,...
67,449,"Expression of the immediate early gene, c-fos,...",Radio Frequency,What is the signal frequency?,Aims: To study the effect of acute exposure to...,900 MHz
68,453,Bone morphogenetic protein expression in newbo...,Research Subject,What animal has been used?,Effects of nonthermal radiofrequency radiation...,rats
69,453,Bone morphogenetic protein expression in newbo...,Radio Frequency,What is the signal frequency?,Effects of nonthermal radiofrequency radiation...,9.4 GHz
70,455,IRIDIUM exposure increases c-fos expression in...,Research Subject,What animal has been used?,With the rapid development of wireless communi...,mice


In [469]:
# text_len : Number of answers
def get_text(text_len, answers):
  texts = []
  for i in range(text_len):
    texts.append(answers[i]['Answer'])
  return texts

In [470]:
test_context.head(5)

,Index,Paper,Category,Question,Abstract,Answer
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...


In [471]:
# A function definition converting answers into list type for easy handling
# text_len : Number of answers
def get_text(answers):
  texts = []
  texts.append(answers)
  return texts

In [472]:
# Running the above function
# Taking the num of answers & answers as input, and converting them into List type
# get_text(1, test_context['Answer'][0])
get_text(test_context['Answer'][0])

['Rats']

In [473]:
# Doing the above job to all the test dataset
test_context['Answer_list'] = test_context.apply(lambda x: get_text(x['Answer']), axis=1)

In [474]:
test_context.head(5)

,Index,Paper,Category,Question,Abstract,Answer,Answer_list
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats,[Rats]
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz,[900 MHz]
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats,[rats]
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz,[900 MHz]
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...,[pulsed emission with a specific absorption ra...


In [475]:
test_context['Answer_list']

0                                                [Rats]
1                                             [900 MHz]
2                                                [rats]
3                                             [900 MHz]
4     [pulsed emission with a specific absorption ra...
                            ...                        
67                                            [900 MHz]
68                                               [rats]
69                                            [9.4 GHz]
70                                               [mice]
71                                            [1.6-GHz]
Name: Answer_list, Length: 72, dtype: object

In [476]:
# Making Input Data (Test Dataset) for BERT Model -- Embedding
# Utilizaing the above Input model in the past, but without answers!

# Column name of context
FINE_DATA_COLUMN = "Abstract"
# Column name of question
FINE_QUESTION_COLUMN = "Question"
# Column name of answer ('text')
FINE_TEXT = "Answer_list"

In [477]:
# Copied & Pasted, and a bit modified from the above function

# Define a function calculating the length of answer in context dataset all at once
def convert_data(data_df):
  global tokenizer
  indices, segments, target_start, target_end = [], [], [], []

  for i in tqdm(range(len(data_df))):
    # que : List of tokenized question
    que, _ = tokenizer.encode(data_df[FINE_QUESTION_COLUMN][i])
    # doc : List of tokenized context
    doc, _ = tokenizer.encode(data_df[FINE_DATA_COLUMN][i])

    # [CLS] token deleted in context
    doc.pop(0)

    # Length of question & context
    que_len = len(que)
    doc_len = len(doc)
    # 1. Length of question
    # The question is cut by the length of 64
    if que_len > 64:
      que = que[:63]
      que.append(102) # [SEP] token added to make it clear the question block
    # 2. Total length of question and context
    # The total input is cut by the length of 384
    if len(que+doc) > SEQ_LEN:
      while len(que+doc) != SEQ_LEN:
        doc.pop(-1)
      doc.pop(-1)
      doc.append(102)

    # Segment embedding
    # Question : 0 / Context 1 / Padding : 0 (remaining part for short sentences)
        
    ############################
    ###### Segment 예시 ########
    ############################
    
    # question, context, padding
    # 00000000, 1111111, 0000000
    
    segment = [0]*len(que) + [1]*len(doc) + [0]*(SEQ_LEN-len(que)-len(doc))

    # Padding
    if len(que+doc) <= SEQ_LEN:
      while len(que+doc) != SEQ_LEN:
        doc.append(0)

    # Final Input 'ids' (Question + Context)
    ids = que + doc

    # Sliding Part
    texts = data_df[FINE_TEXT][i]
    
    for text_element in texts:
      text = tokenizer.encode(text_element)[0]
      text_slide_len = len(text[1:-1]) # text_slide_len = 8

      for j in range(0, (len(doc))):
        # exist_flag : showing whether it is answerable or not (similar with is_unanswerable in SimpleTransforemr)
        # 0 : No answer / 1 : Having answer
        exist_flag = 0 
        if text[1:-1] == doc[j:j+text_slide_len]: # [0:8]->[1:9]->[2:10]->..->[159:160]
          # Assign the location of answer (start, end)
          ans_start = j + len(que)
          ans_end = j + text_slide_len - 1 + len(que)
          # If matched, exist_flag changed to 1
          exist_flag = 1
          break

      # When no answer case (exist_flag = 0), starting & ending value become SEQ_LEN
      # All the data of starting, ending = 384 (SEQ_LEN) will be deleted from the list
      if exist_flag == 0:
        ans_start = SEQ_LEN
        ans_end = SEQ_LEN

    # Input(ids), Segment saving into list type (indices, segments)
    indices.append(ids)
    segments.append(segment)
    # Starting and ending info saving into list type (target_start, target_end)
    target_start.append(ans_start)
    target_end.append(ans_end)

  # Converting the 4 lists into numpy array
  indices_x = np.array(indices)
  segments = np.array(segments)
  target_start = np.array(target_start)
  target_end = np.array(target_end)

  # The cut part saved in del_list and deleted from data
  del_list = np.where(target_start != SEQ_LEN)[0]
  not_del_list = np.where(target_start == SEQ_LEN)[0]
  indices_x = indices_x[del_list]
  segments = segments[del_list]
  
  target_start = target_start[del_list]
  target_end = target_end[del_list]

  return [indices_x, segments], del_list

In [478]:
# Converting the test dataset into BERT input format
dev_bert_input = convert_data(test_context)

100%|██████████| 72/72 [00:00<00:00, 249.77it/s]


In [479]:
# Format : Token + Segment
dev_bert_input

([array([[  101,  2054,  4111, ...,     0,     0,     0],
         [  101,  2054,  2003, ...,     0,     0,     0],
         [  101,  2054,  4111, ..., 11721,  3676,   102],
         ...,
         [  101,  2054,  2003, ...,  1012,  1006,   102],
         [  101,  2054,  4111, ...,     0,     0,     0],
         [  101,  2054,  2003, ...,     0,     0,     0]]),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 1, 1],
         ...,
         [0, 0, 0, ..., 1, 1, 1],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])],
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71]))

In [480]:
# Cut the part over length of 384 ('del_list')
dev_bert_input, del_list = dev_bert_input[0], dev_bert_input[1]
dev = test_context.iloc[del_list]
dev = test_context.reset_index(drop=True)

In [481]:
dev

,Index,Paper,Category,Question,Abstract,Answer,Answer_list
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats,[Rats]
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz,[900 MHz]
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats,[rats]
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz,[900 MHz]
4,372,Effects of radiofrequency exposure on the GABA...,Other Units of Exposure Level,How much W/kg was used?,The widespread use of cellular phones raises t...,pulsed emission with a specific absorption rat...,[pulsed emission with a specific absorption ra...
...,...,...,...,...,...,...,...
67,449,"Expression of the immediate early gene, c-fos,...",Radio Frequency,What is the signal frequency?,Aims: To study the effect of acute exposure to...,900 MHz,[900 MHz]
68,453,Bone morphogenetic protein expression in newbo...,Research Subject,What animal has been used?,Effects of nonthermal radiofrequency radiation...,rats,[rats]
69,453,Bone morphogenetic protein expression in newbo...,Radio Frequency,What is the signal frequency?,Effects of nonthermal radiofrequency radiation...,9.4 GHz,[9.4 GHz]
70,455,IRIDIUM exposure increases c-fos expression in...,Research Subject,What animal has been used?,With the rapid development of wireless communi...,mice,[mice]


In [482]:
# Save the only tokenized sentences input (1~10527)
# Without the segment part
indexes = dev_bert_input[0]

In [483]:
indexes

array([[  101,  2054,  4111, ...,     0,     0,     0],
       [  101,  2054,  2003, ...,     0,     0,     0],
       [  101,  2054,  4111, ..., 11721,  3676,   102],
       ...,
       [  101,  2054,  2003, ...,  1012,  1006,   102],
       [  101,  2054,  4111, ...,     0,     0,     0],
       [  101,  2054,  2003, ...,     0,     0,     0]])

In [484]:
# BERT prediction start to all the test dataset (10,527 * 384 tokens * 787 features), taking about 2 mins
bert_predictions = bert_model.predict(dev_bert_input)

In [485]:
bert_predictions

[array([[8.33515855e-07, 6.41706492e-06, 9.95121127e-07, ...,
         8.78516460e-07, 8.77021364e-07, 8.76592367e-07],
        [9.30958421e-08, 1.65816672e-07, 9.70450600e-08, ...,
         9.18691967e-08, 9.21201107e-08, 9.22358012e-08],
        [1.36415360e-06, 1.02564103e-04, 4.30890759e-06, ...,
         1.56488557e-06, 1.60175580e-06, 1.60558852e-06],
        ...,
        [8.25969124e-08, 9.20755170e-08, 8.92646170e-08, ...,
         9.07327831e-08, 8.35777030e-08, 9.10357940e-08],
        [1.97030863e-06, 2.65223716e-05, 2.96403300e-06, ...,
         1.99256669e-06, 1.99163378e-06, 1.99001829e-06],
        [1.19208622e-07, 1.09767683e-07, 1.27985643e-07, ...,
         1.14655386e-07, 1.14858501e-07, 1.14944527e-07]], dtype=float32),
 array([[8.8161556e-07, 1.3215661e-06, 8.6560141e-07, ..., 1.0125283e-06,
         1.0104352e-06, 1.0103648e-06],
        [3.8212355e-07, 3.6744186e-07, 3.9342416e-07, ..., 3.6077640e-07,
         3.6133937e-07, 3.6179247e-07],
        [2.0332345e-06

In [486]:
# Among 384 tokens, selecting the one having the largest probability
start_indexes = np.argmax(bert_predictions[0], axis=-1)
end_indexes = np.argmax(bert_predictions[1], axis=-1)

In [487]:
# Delete some non-sense case 
# e.g. position of starting token (start_indexes) is bigger than the ending token (end_indexes)
not_del_list = np.where(start_indexes <= end_indexes)[0]

In [488]:
not_del_list.shape

(71,)

In [489]:
start_indexes = start_indexes[not_del_list]
end_indexes = end_indexes[not_del_list]
indexes = indexes[not_del_list]

In [490]:
# Positions of starting tokens
start_indexes[0:100]

array([108, 121, 100, 241,  47,  43, 118, 125, 137, 160,  90,  38, 203,
        43,  41,  77, 101, 101, 127,  80, 313, 103, 225, 163,  45, 101,
        91,  69,  31,  99,  22,  63,  62,  73, 115, 101, 163,  81,  90,
       108,  54, 164,  68,  49,  69,  99,  98,  73, 294,  57,  68,  52,
        48,  63,  97,  63,  71,   8,   9,  19,  32,  46,  97,  66,  83,
        57,  54,  82,  92,  74,  77])

In [491]:
# Positions of end tokens
end_indexes[0:100]

array([108, 122, 100, 242,  51,  45, 120, 125, 145, 169,  90,  42, 208,
        47,  41,  79, 108, 105, 128,  85, 315, 108, 225, 164,  49, 101,
        94,  69,  34,  99,  26,  69,  62,  74, 115, 103, 168,  86,  92,
       113,  55, 173,  71,  49,  74, 105,  98,  74, 301,  63,  69,  54,
        52,  74, 106,  63, 100,   8,  15,  19,  33,  51,  97,  68,  97,
        57,  55,  82,  95,  74,  81])

In [492]:
# After deleting the non-sense case, rearranging dataset
dev = dev.iloc[not_del_list].reset_index(drop=True)

In [493]:
dev

,Index,Paper,Category,Question,Abstract,Answer,Answer_list
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Research Subject,What animal has been used?,Background: The widespread use of mobile phone...,Rats,[Rats]
1,371,Ginkgo biloba prevents mobile phone-induced ox...,Radio Frequency,What is the signal frequency?,Background: The widespread use of mobile phone...,900 MHz,[900 MHz]
2,372,Effects of radiofrequency exposure on the GABA...,Research Subject,What animal has been used?,The widespread use of cellular phones raises t...,rats,[rats]
3,372,Effects of radiofrequency exposure on the GABA...,Radio Frequency,What is the signal frequency?,The widespread use of cellular phones raises t...,900 MHz,[900 MHz]
4,376,Long-term exposure to electromagnetic radiatio...,Research Subject,What animal has been used?,The aim of the present study was the investiga...,BALB/c mice,[BALB/c mice]
...,...,...,...,...,...,...,...
66,449,"Expression of the immediate early gene, c-fos,...",Radio Frequency,What is the signal frequency?,Aims: To study the effect of acute exposure to...,900 MHz,[900 MHz]
67,453,Bone morphogenetic protein expression in newbo...,Research Subject,What animal has been used?,Effects of nonthermal radiofrequency radiation...,rats,[rats]
68,453,Bone morphogenetic protein expression in newbo...,Radio Frequency,What is the signal frequency?,Effects of nonthermal radiofrequency radiation...,9.4 GHz,[9.4 GHz]
69,455,IRIDIUM exposure increases c-fos expression in...,Research Subject,What animal has been used?,With the rapid development of wireless communi...,mice,[mice]


In [494]:
# Getting the length of test data
length = len(dev)

In [495]:
length

71

In [496]:
sentences = []
untokenized = []

for j in range(len(start_indexes)):
  sentence = []
  # Saving each tokenized word into the list of sentence (sentence = [])
  for i in range(start_indexes[j], end_indexes[j]+1):
    token_based_word = reverse_token_dict[indexes[j][i]]
    sentence.append(token_based_word)
  sentence_string = ""

  for w in sentence:
    # Special token ## delete, if a token starts with ##
    if w.startswith("##"):
      w = w.replace("##", "")
    # If a token has no ##, a space added to the word
    else:
      w = " " + w
    # Putting together all the tokens in list format
    sentence_string += w
  
  # If senetence_string starts with a space (" "), delete the space
  if sentence_string.startswith(" "):
    sentence_string = "" + sentence_string[1:]
  
  # After putting together all the list of tokens, and assigning it in the list of the 'Untokenized'
  untokenized.append(sentence_string)
  sentences.append(sentence)

In [497]:
print(sentences[:30])
print('\n')
print(untokenized[:30])

[['rats'], ['900', 'mhz'], ['rats'], ['900', 'mhz'], ['bal', '##b', '/', 'c', 'mice'], ['forty', '-', 'eight'], ['wi', '##star', 'rats'], ['96'], ['900', 'mhz', ',', '1800', 'mhz', 'and', '245', '##0', 'mhz'], ['5', '.', '84', '×', '10', '-', '4', 'w', '/', 'kg'], ['rats'], ['gs', '##m', '-', '1800', 'mhz'], ['3', '.', '22', 'w', '/', 'kg'], ['wi', '##star', 'al', '##bino', 'rats'], ['24'], ['210', '##0', 'mhz'], ['c', '##57', '##bl', '/', '6', 'adult', 'male', 'mice'], ['c', '##57', '##bl', '/', '6'], ['1800', 'mhz'], ['c', '##57', '##bl', '/', '6', 'mice'], ['83', '##5', 'mhz'], ['4', '.', '0', 'w', '/', 'kg'], ['rats'], ['900', 'mhz'], ['bal', '##b', '/', 'c', 'mice'], ['twelve'], ['900', 'and', '1800', 'mhz'], ['rats'], ['1800', '##m', '##h', '##z'], ['rats']]


['rats', '900 mhz', 'rats', '900 mhz', 'balb / c mice', 'forty - eight', 'wistar rats', '96', '900 mhz , 1800 mhz and 2450 mhz', '5 . 84 × 10 - 4 w / kg', 'rats', 'gsm - 1800 mhz', '3 . 22 w / kg', 'wistar albino rats', '24

In [498]:
# Converting the answers into list format
dev_answers = []
for i in range(length):
  dev_answer = []
  texts_dict = test_context['Answer']

  for j in range(len(texts_dict)):
    dev_answer.append(texts_dict[j])
  dev_answers.append(dev_answer)

In [499]:
dev_answers[0]

['Rats',
 '900 MHz',
 'rats',
 '900 MHz',
 'pulsed emission with a specific absorption rate (SAR) of 4 W/kg and continuous emission with high SAR (32 W/kg)',
 'BALB/c mice',
 'Forty-eight',
 'Wistar rats',
 '96',
 '900 MHz, 1800 MHz and 2450 MHz',
 '5.84 × 10-4 W/kg, 5.94 × 10-4 W/kg and 6.4 × 10-4 W/kg',
 'rats',
 '1800 MHz',
 '3.22 W/kg',
 'Wistar albino rats',
 '24',
 '2100 MHz',
 'mice',
 'whole-body exposed (nExp = 8) for 2 hr to GSM 1800 MHz mobile phone radiation at an average electric field intensity range of 4.3-17.5 V/m or sham-exposed (nSE = 8)',
 '1800 MHz',
 'C57BL/6 mice',
 '835 MHz',
 '4.0 W/kg',
 'Rats',
 '900 MHz',
 'BALB/c mice',
 'twelve',
 '900 and 1800 MHz',
 'rats',
 '1800MHz',
 'Rats',
 '900 and 2100 MHz',
 'Sprague-Dawley rats',
 '20',
 '900 MHz',
 'mice',
 '835 MHz',
 '4.0 W/kg',
 'C57BL/6 mice',
 '835 MHz',
 '4.0 W/kg',
 'Chicken eggs',
 '450 eggs per treatment in three successive rounds per treatment',
 'GSM (1.8 GHz), DECT (1.88 GHz), UMTS (2.1 GHz), and WLA

In [500]:
# Tokenize the answers
dev_tokens = []
for i in dev_answers:
  dev_tokened = []
  for j in i:
    temp_token = tokenizer.tokenize(j)
    # Tokenize an answer
    temp_token.pop(0)
    # [CLS] elimination
    temp_token.pop(-1)
    # [SEP] elimination
    dev_tokened.append(temp_token)
  dev_tokens.append(dev_tokened)

In [501]:
print(dev_tokens[:1])

[[['rats'], ['900', 'mhz'], ['rats'], ['900', 'mhz'], ['pulsed', 'emission', 'with', 'a', 'specific', 'absorption', 'rate', '(', 'sar', ')', 'of', '4', 'w', '/', 'kg', 'and', 'continuous', 'emission', 'with', 'high', 'sar', '(', '32', 'w', '/', 'kg', ')'], ['bal', '##b', '/', 'c', 'mice'], ['forty', '-', 'eight'], ['wi', '##star', 'rats'], ['96'], ['900', 'mhz', ',', '1800', 'mhz', 'and', '245', '##0', 'mhz'], ['5', '.', '84', '×', '10', '-', '4', 'w', '/', 'kg', ',', '5', '.', '94', '×', '10', '-', '4', 'w', '/', 'kg', 'and', '6', '.', '4', '×', '10', '-', '4', 'w', '/', 'kg'], ['rats'], ['1800', 'mhz'], ['3', '.', '22', 'w', '/', 'kg'], ['wi', '##star', 'al', '##bino', 'rats'], ['24'], ['210', '##0', 'mhz'], ['mice'], ['whole', '-', 'body', 'exposed', '(', 'ne', '##x', '##p', '=', '8', ')', 'for', '2', 'hr', 'to', 'gs', '##m', '1800', 'mhz', 'mobile', 'phone', 'radiation', 'at', 'an', 'average', 'electric', 'field', 'intensity', 'range', 'of', '4', '.', '3', '-', '17', '.', '5', 'v',

In [502]:
# Converting tokenized answers into sentences
# And putting all together
dev_answer_lists = []
for dev_answers in dev_tokens:
  dev_answer_list = []
  for dev_answer in dev_answers:
    dev_answer_string = " ".join(dev_answer)
    dev_answer_list.append(dev_answer_string)
  dev_answer_lists.append(dev_answer_list)

In [503]:
print(dev_answer_lists[:1])

[['rats', '900 mhz', 'rats', '900 mhz', 'pulsed emission with a specific absorption rate ( sar ) of 4 w / kg and continuous emission with high sar ( 32 w / kg )', 'bal ##b / c mice', 'forty - eight', 'wi ##star rats', '96', '900 mhz , 1800 mhz and 245 ##0 mhz', '5 . 84 × 10 - 4 w / kg , 5 . 94 × 10 - 4 w / kg and 6 . 4 × 10 - 4 w / kg', 'rats', '1800 mhz', '3 . 22 w / kg', 'wi ##star al ##bino rats', '24', '210 ##0 mhz', 'mice', 'whole - body exposed ( ne ##x ##p = 8 ) for 2 hr to gs ##m 1800 mhz mobile phone radiation at an average electric field intensity range of 4 . 3 - 17 . 5 v / m or sham - exposed ( ns ##e = 8 )', '1800 mhz', 'c ##57 ##bl / 6 mice', '83 ##5 mhz', '4 . 0 w / kg', 'rats', '900 mhz', 'bal ##b / c mice', 'twelve', '900 and 1800 mhz', 'rats', '1800 ##m ##h ##z', 'rats', '900 and 210 ##0 mhz', 'sp ##rag ##ue - da ##wley rats', '20', '900 mhz', 'mice', '83 ##5 mhz', '4 . 0 w / kg', 'c ##57 ##bl / 6 mice', '83 ##5 mhz', '4 . 0 w / kg', 'chicken eggs', '450 eggs per trea

In [504]:
# Untokenizing (including deleting " ##" etc)
dev_strings_end = []
for dev_strings in dev_answer_lists:
  dev_strings_processed = []
  for dev_string in dev_strings:
    dev_string = dev_string.replace(" ##", "")
    dev_strings_processed.append(dev_string)
  dev_strings_end.append(dev_strings_processed)

In [505]:
dev_answers = dev_strings_end

In [506]:
print(dev_answers[:1])

[['rats', '900 mhz', 'rats', '900 mhz', 'pulsed emission with a specific absorption rate ( sar ) of 4 w / kg and continuous emission with high sar ( 32 w / kg )', 'balb / c mice', 'forty - eight', 'wistar rats', '96', '900 mhz , 1800 mhz and 2450 mhz', '5 . 84 × 10 - 4 w / kg , 5 . 94 × 10 - 4 w / kg and 6 . 4 × 10 - 4 w / kg', 'rats', '1800 mhz', '3 . 22 w / kg', 'wistar albino rats', '24', '2100 mhz', 'mice', 'whole - body exposed ( nexp = 8 ) for 2 hr to gsm 1800 mhz mobile phone radiation at an average electric field intensity range of 4 . 3 - 17 . 5 v / m or sham - exposed ( nse = 8 )', '1800 mhz', 'c57bl / 6 mice', '835 mhz', '4 . 0 w / kg', 'rats', '900 mhz', 'balb / c mice', 'twelve', '900 and 1800 mhz', 'rats', '1800mhz', 'rats', '900 and 2100 mhz', 'sprague - dawley rats', '20', '900 mhz', 'mice', '835 mhz', '4 . 0 w / kg', 'c57bl / 6 mice', '835 mhz', '4 . 0 w / kg', 'chicken eggs', '450 eggs per treatment in three successive rounds per treatment', 'gsm ( 1 . 8 ghz ) , dect 

# 2. Fine-tuning Evaluation - F1 & EM

In [507]:
# F1 & EM function defined by myself
from collections import Counter
import string

# In normalize step, doing some works such as converting words into lower case,
# and deleting punctuations, clearing unnecessary spaces etc
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)
  
  def white_space_fix(text):
    return ' '.join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()
  
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def exact_match_score(prediction, ground_truth):
  return (normalize_answer(prediction) == normalize_answer(ground_truth))

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
  scores_for_ground_truths = []
  for ground_truth in ground_truths:
    score = metric_fn(prediction, ground_truth)
    scores_for_ground_truths.append(score)
  return max(scores_for_ground_truths)

In [508]:
# Calculating F1 Score
# Theoretical Max : 88%
f1_sum = 0

for i in range(len(untokenized)):
  f1 = metric_max_over_ground_truths(f1_score, untokenized[i], dev_answers[i])
  f1_sum += f1
print("f1 score: ", f1_sum/length)

f1 score:  0.963148588500701


In [509]:
# Calculating EM score
# Theoretical Max : 80%
EM_sum = 0

for i in range(len(untokenized)):
  EM = metric_max_over_ground_truths(exact_match_score, untokenized[i], dev_answers[i])
  EM_sum += EM
print("EM score: ", EM_sum/length)

EM score:  0.8732394366197183


# 3. Test for SW Development

## *1. Without Null Data*

In [523]:
def convert_data(data_df):
    global tokenizer
    indices, segments = [], []

  
    que, _ = tokenizer.encode(data_df[FINE_QUESTION_COLUMN])
    doc, _ = tokenizer.encode(data_df[FINE_DATA_COLUMN])
    doc.pop(0)

    que_len = len(que)
    doc_len = len(doc)

    if que_len > 64:
      que = que[:63]
      que.append(102)
        
    if len(que+doc) > SEQ_LEN:
      while len(que+doc) != SEQ_LEN:
        doc.pop(-1)

      doc.pop(-1)
      doc.append(102)

    segment = [0]*len(que) + [1]*len(doc) + [0]*(SEQ_LEN-len(que)-len(doc))
    if len(que + doc) <= SEQ_LEN:
      while len(que+doc) != SEQ_LEN:
        doc.append(0)

    ids = que + doc


    indices.append(ids)
    segments.append(segment)
    indices = np.array(indices)
    segments = np.array(segments)
    return [indices, segments]

In [560]:
def predict_letter(df):
  
  test_input = convert_data(df)
  test_start, test_end = bert_model.predict(test_input)
  
  indexes = test_input[0].tolist()[0]
  start = np.argmax(test_start, axis=1).item()
  end = np.argmax(test_end, axis=1).item()
  start_tok = indexes[start]
  end_tok = indexes[end]

  print("Context : ", end = " ")
  
  def split_text(text, n):
    for line in text.splitlines():
        while len(line) > n:
           x, line = line[:n], line[n:]
           yield x
        yield line
  
  for line in split_text(df['Abstract'], 150):
    print(line)

  print("-"*100)
  print("Question : ", df['Question'])
  # print("Tokinized Answer : ", end = " ") # Tokenized Answer
  sentences = []
  
  for i in range(start, end+1):
    token_based_word = reverse_token_dict[indexes[i]]
    sentences.append(token_based_word)
    # print(token_based_word, end= " ")
  
  print("Machine Prediction : ", end = "") # Untokenized Answer (final Machine Prediction)
  for w in sentences:
    if w.startswith("##"):
      w = w.replace("##", "")
    else:
      w = " " + w
    
    print(w, end="")
  # print("")

In [561]:
for i in range(71):
  
  print("Test Data [", i+1, "/ 71 ]")
  print("-"*100)

  answers = dev['Answer'][i]
  predict_letter(dev.iloc[i])
  print("")
  print("ANSWER : ", answers)
  print("")
  print("")

Test Data [ 1 / 71 ]
----------------------------------------------------------------------------------------------------
Context :  Background: The widespread use of mobile phones (MP) in recent years has raised the research activities in many countries to determine the consequence
s of exposure to the low-intensity electromagnetic radiation (EMR) of mobile phones. Since several experimental studies suggest a role of reactive oxy
gen species (ROS) in EMR-induced oxidative damage in tissues, in this study, we investigated the effect of Ginkgo biloba (Gb) on MP-induced oxidative 
damage in brain tissue of rats.

Methods: Rats (EMR+) were exposed to 900 MHz EMR from MP for 7 days (1 h/day). In the EMR+Gb groups, rats were exposed to EMR and pretreated with Gb. 
Control and Gb-administrated groups were produced by turning off the mobile phone while the animals were in the same exposure conditions. Subsequently
, oxidative stress markers and pathological changes in brain tissue were examin

## *2. With Null Data*

In [562]:
# Test dataset loading
import pandas as pd
test_context = pd.read_excel('Test_QA_OneByOne.xlsx')
test_question = pd.read_excel('Question.xlsx')

In [563]:
test_context

,Index,Paper,Abstract,Target,Method In-Vivo,Method In-Vitro,Method Others
0,371,Ginkgo biloba prevents mobile phone-induced ox...,Background: The widespread use of mobile phone...,1,1,0,0
1,372,Effects of radiofrequency exposure on the GABA...,The widespread use of cellular phones raises t...,1,1,0,0
2,376,Long-term exposure to electromagnetic radiatio...,The aim of the present study was the investiga...,1,1,0,0
3,377,Effect of mobile phone signal radiation on epi...,"Exponential increase in mobile phone uses, giv...",1,1,0,0
4,378,Effects of a Single Head Exposure to GSM-1800 ...,Mobile communications are propagated by electr...,1,1,0,0
5,379,Melatonin Modulates NMDA-Receptor 2B/Calpain-1...,Aim: To investigate the potential protective e...,1,1,0,0
6,382,Hippocampal lipidome and transcriptome profile...,Background: The widespread use of wireless dev...,1,1,0,0
7,383,Exposure to 835 MHz RF-EMF decreases the expre...,The exponential increase in the use of mobile ...,1,1,0,0
8,386,Effects of 2G mobile phone exposure on both be...,Mobile communications are expanded day by day ...,1,1,0,0
9,387,"Evaluation of bax, bcl-2, p21 and p53 genes ex...",Objectives: The increasing rate of over using ...,1,1,0,0


In [564]:
df_temp = test_context['Abstract']
test_context = df_temp.to_frame()

In [565]:
test_context

,Abstract
0,Background: The widespread use of mobile phone...
1,The widespread use of cellular phones raises t...
2,The aim of the present study was the investiga...
3,"Exponential increase in mobile phone uses, giv..."
4,Mobile communications are propagated by electr...
5,Aim: To investigate the potential protective e...
6,Background: The widespread use of wireless dev...
7,The exponential increase in the use of mobile ...
8,Mobile communications are expanded day by day ...
9,Objectives: The increasing rate of over using ...


In [566]:
test_question

,Question
0,What animal has been used?
1,How many animals were used?
2,What is the signal frequency?
3,How much W/kg was used?


In [567]:
len(test_context)

29

In [568]:
# Copied & Pasted, and a bit modified from the above function
# Define a function calculating the length of answer in context dataset all at once
def convert_data(test_context, test_question):
  global tokenizer
  indices, segments = [], []

  que, _ = tokenizer.encode(test_question['Question'])
  doc, _ = tokenizer.encode(test_context['Abstract'])
  doc.pop(0)

  que_len = len(que)
  doc_len = len(doc)

  # 1. Length of question
  # The question is cut by the length of 64
  if que_len > 64:
    que = que[:63]
    que.append(102) # [SEP] token added to make it clear the question block
  
  # 2. Total length of question and context
  # The total input is cut by the length of 384
  if len(que+doc) > SEQ_LEN:
    while len(que+doc) != SEQ_LEN:
      doc.pop(-1)
    doc.pop(-1)
    doc.append(102)

    # Segment embedding
    # Question : 0 / Context 1 / Padding : 0 (remaining part for short sentences)
        
    ############################
    ###### Segment 예시 ########
    ############################
    
    # question, context, padding
    # 00000000, 1111111, 0000000
    
  segment = [0]*len(que) + [1]*len(doc) + [0]*(SEQ_LEN-len(que)-len(doc))

  # Padding
  if len(que+doc) <= SEQ_LEN:
    while len(que+doc) != SEQ_LEN:
      doc.append(0)

  # Final Input 'ids' (Question + Context)
  ids = que + doc

  # Input(ids), Segment saving into list type (indices, segments)
  indices.append(ids)
  segments.append(segment)

  # Converting the 4 lists into numpy array
  indices = np.array(indices)
  segments = np.array(segments)
  
  # print(indices, segments)
  return [indices, segments]

In [571]:
def predict_letter(df_context, df_question):
  test_input = convert_data(df_context, df_question)
  test_start, test_end = bert_model.predict(test_input)

  indexes = test_input[0].tolist()[0]
  start = np.argmax(test_start, axis=1).item()
  end = np.argmax(test_end, axis=1).item()
  start_tok = indexes[start]
  end_tok = indexes[end]

  # # print("Context: ", end = " ")
  # print("Context")
  # print("-"*100)

  # def split_text(text, n):
  #   for line in text.splitlines():
  #     while len(line) > n:
  #       x, line = line[:n], line[n:]
  #       yield x
  #     yield line
  
  # for line in split_text(df_context['Abstract'], 100):
  #   print(line)

  # print("-"*100)
  
  print("Question: ", df_question['Question'])
  print("ANSWER: ", end = " ")
  # print("\n")
  sentences = []

  for i in range(start, end+1):
    token_based_word = reverse_token_dict[indexes[i]]
    sentences.append(token_based_word)
    # print(token_based_word, end= " ")
  # print("\n")
  # print("Untokenized Answer: ", end= "")
  
  for w in sentences:
    if w.startswith("##"):
      w = w.replace("##", "")
    else:
      w = " " + w
    print(w, end="")
  # print("")

In [574]:
for i in range(29):
  
  print("Context [", i+1, "/ 29 ]")
  print("-"*100)
  def split_text(text, n):
    for line in text.splitlines():
      while len(line) > n:
        x, line = line[:n], line[n:]
        yield x
      yield line
  
  for line in split_text(test_context.iloc[i]['Abstract'], 100):
    print(line)

  print("-"*100)

  for j in range(4):
  # answers = test['answers'][i]
    predict_letter(test_context.iloc[i], test_question.iloc[j])
    print("")
    print("")
  print("")
  print("")

Context [ 1 / 29 ]
----------------------------------------------------------------------------------------------------
Background: The widespread use of mobile phones (MP) in recent years has raised the research activit
ies in many countries to determine the consequences of exposure to the low-intensity electromagnetic
 radiation (EMR) of mobile phones. Since several experimental studies suggest a role of reactive oxy
gen species (ROS) in EMR-induced oxidative damage in tissues, in this study, we investigated the eff
ect of Ginkgo biloba (Gb) on MP-induced oxidative damage in brain tissue of rats.

Methods: Rats (EMR+) were exposed to 900 MHz EMR from MP for 7 days (1 h/day). In the EMR+Gb groups,
 rats were exposed to EMR and pretreated with Gb. Control and Gb-administrated groups were produced 
by turning off the mobile phone while the animals were in the same exposure conditions. Subsequently
, oxidative stress markers and pathological changes in brain tissue were examined for each